<a href="https://colab.research.google.com/github/tiendatngcs/NLP/blob/master/Data_cleaning_Vietnamese_news_topic_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pickle as pkl
import string
import random

In [3]:
with open("/content/drive/My Drive/corpus.pkl", "rb") as infile:
  corpus_dict = pkl.load(infile)

In [4]:
topic_list = [
              "business",
              "entertainment",
              "sports",
              "laws",
              "education",
              "health",
              "life",
              "travel",
              "science",
              "digital",
              "vehicles",
              "comedy",
]
topic_dict = dict(enumerate(topic_list))

In [7]:
print("Topic: Number of articles")
for (key, value) in corpus_dict.items():
  print("{}: {}".format(topic_dict[key], len(value)))

Topic: Number of articles
business: 550
entertainment: 499
sports: 560
laws: 43
education: 35
health: 520
life: 545
travel: 468
science: 33
digital: 500
vehicles: 44
comedy: 35


In [12]:
corpus_dict[3]

['"Mẹ đừng có thương người nhầm chỗ nữa", con trai bà Tần ngồi cuối phòng xử án của TAND Hà Nội vò đầu khẽ nói khi nghe thấy bà xin HĐXX không bỏ tù bị cáo, ngày 20/8. Bà giáo già vuốt lại nếp tóc bạc trắng đầu, quay sang con trai phân trần: "Em nó còn 3 đứa con nhỏ, mẹ không đành". Sáng 21/8, Giang, 39 tuổi, cựu giám đốc Công ty TNHH MTV chứng khoán ngân hàng Đông Á (DAS) bị TAND Hà Nội tuyên phạt 20 năm tù về hai tội Thao túng giá chứng khoán và Lừa đảo chiếm đoạt tài sản. Trong hai ngày xét xử, khi đi qua cô giáo cũ sau hai năm tránh mặt, trốn nợ rồi bị bắt, Giang không chút biểu cảm. Trước đó, họ từng nhiều năm coi nhau như người nhà. Vợ chồng bà Tần bị lừa tiền vẫn xin HĐXX cho bị cáo Giang hưởng khoan hồng. Ảnh: Lam Vân 50 năm trước, bà Tần tốt nghiệp ngành sư phạm Văn. Trong khoá học sinh đầu tiên bà làm chủ nhiệm suốt 4 năm có mẹ của Giang. Bà Tần cưng cô trò thông minh này, ngày thường đến chơi, ngày cưới đến dự. Năm 1981, ngày Giang ra đời, bà Tần không có xe đạp, vẫn đi bộ m

In [14]:
selected_corpus_list = []
selected_topic_list = []
for (key, value) in corpus_dict.items():
  if len(value)> 300:
    selected_topic_list.append(topic_dict[key])
    selected_corpus_list.append(value)
selected_corpus_dict = dict(enumerate(selected_corpus_list))

We select classes that has decent amount of data for training and testing.

In [17]:
selected_corpus_dict.keys()
selected_topic_list

['business', 'entertainment', 'sports', 'health', 'life', 'travel', 'digital']

Let's select 300 articles from each selected topics for training. The rest are for testing.

In [ ]:
def clean_text(corpus_dict: dict):
  

In [19]:
def train_test_split(corpus_dict: dict,
                     train_size: int,
                     shuffle: bool = False,
                     random_seed: int = None,
                     join_strings: bool = True,
                     clean_text: bool = True):
  '''
  This method performs three tasks:
  * Join the list of strings for each class.
  * Clean the text data for each class.
  * Split the text data into training and testing dicts.
  '''
  #join
  for (key, value) in corpus_dict.items():
    corpus_dict[key] = ' '.join(value)
  #clesn text data
  